In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import copy
import ast
import sys
sys.path.append('../../dash_gaia_orbits/')
from dash_gaia_orbits import StarClusterData, StarClusterCollection, StarClusters3DPlotter
df = pd.read_csv('/Users/cam/Downloads/clusters_sne_times.csv')
df = df[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x_helio', 'y_helio', 'z_helio', 'n_stars', 'mass_50', 'n_sne_50', 'time', 'count']]
df = df.rename(columns={'x_helio' : 'x', 'y_helio' : 'y', 'z_helio' : 'z', 'count' : 'sne_count', 'time' : 'sne_time'})
df = df[df['n_sne_50'] >= 1]
df['sne_time'] = df['sne_time'].apply(lambda x: np.fromstring(x.strip('[]'), sep=' '))
df['sne_count'] = df['sne_count'].apply(lambda x: np.fromstring(x.strip('[]'), sep=' '))

df_copy = copy.deepcopy(df)
df_copy.drop(columns=['sne_count', 'sne_time'], inplace=True)

A new version of galpy (1.10.0) is available, please upgrade using pip/conda/... to get the latest features and bug fixes!


In [17]:
df.loc[df['name'] == 'UPK_640'][['n_sne_50', 'mass_50']]

,n_sne_50,mass_50
249,1.0,538.879143


In [14]:
1825/100

18.25

In [8]:
df_young = df[df['age_myr'] < 20]
df_young.n_sne_50.sum()/20

3.65

In [47]:
time_int = np.round(np.arange(0., -61, -1), 1)
c = StarClusterData(df_copy, data_name = 'Clusters', min_size = 0, max_size = 6, color = 'gray', opacity = 1, marker_style = 'circle', show_tracks = False)
c_copy = StarClusterData(df, data_name = 'Clusters', min_size = 0, max_size = 6, color = 'gray', opacity = 1, marker_style = 'circle', show_tracks = False)
c_copy.integrate_orbits(time=time_int)
df_int = c_copy.df_int

In [48]:
clusters = df_int.groupby('name')

x_sne = []
y_sne = []
z_sne = []
weight = []

for cluster_name, cluster in clusters:
    cluster_sne_data = df.loc[df['name'] == cluster_name]
    #sne_times = -1*np.round(np.array(cluster_sne_data['sne_time'].values[0]))/2
    sne_times = -1*np.round(np.array(cluster_sne_data['sne_time'].values[0]), 0)
    sne_counts = np.array(cluster_sne_data['sne_count'].values[0])
    for sne_t, sne_c in zip(sne_times, sne_counts):
        x_t = np.array(cluster['x'].loc[cluster['time'] == sne_t].values)
        y_t = np.array(cluster['y'].loc[cluster['time'] == sne_t].values)
        z_t = np.array(cluster['z'].loc[cluster['time'] == sne_t].values)
        if len(x_t) != 0:
            x_sne.append(x_t[0])
            y_sne.append(y_t[0])
            z_sne.append(z_t[0])
            weight.append(sne_c)

In [49]:
scs = StarClusterCollection([c])

In [50]:
scatter_sne = go.Scatter3d(x=x_sne, y=y_sne, z=z_sne, mode='markers', marker=dict(size=5, color=weight, opacity=.08, cmax = .05), name='SNe')
time_show_sne = [0]

In [51]:
xyz_widths = (1000, 1000, 300)

plot_3d = StarClusters3DPlotter(
    data_collection = scs, 
    figure_title=None,
    xyz_widths = xyz_widths, 
    #xyz_ranges=xyz_ranges,
    figure_theme = 'dark', 
    trace_grouping_dict=None)

save_name = '/Users/cam/Downloads/sne_locations.html'
fig3d = plot_3d.generate_3d_plot(
    time = time_int,
    show = False, 
    save_name =save_name, 
    static_traces = [scatter_sne], 
    static_traces_times = [[time_int]],
)